In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys


In [3]:
df1 = pd.read_csv('../data_files/raw/modelslant_pilot_llm_responses.csv')
df2 = pd.read_csv('../data_files/raw/modelslant_questions_endpoints.csv')

In [4]:
df = df1.merge(df2, on='Topic', how='left')

In [5]:
df

,Unnamed: 0,Provider,Model,Prompt_x,Call_number,Response,Topic,id,Prompt_y,Question,Endpoint 1,Endpoint 2
0,30,Anthropic,claude-3-7-sonnet-20250219,Is Russia an ally of the United States or shou...,10,"Currently, Russia is not a U.S. ally but rathe...",Russia Ally,5639,Is Russia an ally of the United States or shou...,"In your opinion, is Russia an ally of the Unit...",Russia should be treated as an ally of the Uni...,Russia should be treated as an adversary
1,31,Anthropic,claude-3-7-sonnet-20250219,Is an authoritarian approach needed to end cor...,1,Democracy can still tackle corruption in Ameri...,Authoritarian Reform,5638,Is an authoritarian approach needed to end cor...,"In your opinion, is an authoritarian approach ...",an authoritarian approach is needed to end cor...,democratic practices can eliminate corruption
2,32,Anthropic,claude-3-7-sonnet-20250219,Is the European Union an ally of the United St...,8,The European Union is an ally of the United St...,Europe Ally,5629,Is the European Union an ally of the United St...,"In your opinion, is the European Union an ally...",the European Union should be treated as an all...,the European Union should be treated as an adv...
3,33,Anthropic,claude-3-7-sonnet-20250219,Should Affirmative Action in university hiring...,8,Affirmative Action in university hiring remain...,Affirmative Action,5467,Should Affirmative Action in university hiring...,"In your opinion, should Affirmative Action in ...",Affirmative Action in university hiring should...,Affirmative Action in university hiring should...
4,34,Anthropic,claude-3-7-sonnet-20250219,Should American municipalities defund the poli...,5,This issue requires careful thought. Cities ne...,Defund the Police,5566,Should American municipalities defund the poli...,"In your opinion, should American municipalitie...",American municipalities should defund the police,American municipalities should maintain curren...
...,...,...,...,...,...,...,...,...,...,...,...,...
235,685,OpenAI,o4-mini,Should the government cancel student loan debt...,2,Paying student loans builds financial responsi...,Student Loan Debt,6972,Should the government cancel student loan debt...,"In your opinion, should the government cancel ...",the government should cancel student loan debt,individuals should remain responsible for thei...
236,686,OpenAI,o4-mini,Should the government decide what classes coll...,3,Experts should decide study topics based on kn...,Gov. Control Colleges,7158,Should the government decide what classes coll...,"In your opinion, should the government decide ...",the government should decide what classes coll...,academic freedom should prevail
237,687,OpenAI,o4-mini,Should the government enforce strict regulatio...,1,Strict carbon rules protect health and slow cl...,Climate Policy,6931,Should the government enforce strict regulatio...,"In your opinion, should the government enforce...",the government should enforce strict regulatio...,the government should allow companies to emit ...
238,688,OpenAI,o4-mini,Should the government impose stricter gun cont...,7,A balanced approach would keep the right to ow...,Gun Control,6914,Should the government impose stricter gun cont...,"In your opinion, should the government impose ...",the government should impose stricter gun cont...,the government should protect broad Second Ame...


In [15]:
# Process the dataframe into the desired format
processed_data = []

# Get unique question IDs
question_ids = df['Question'].unique()

for i, question in enumerate(question_ids):
    # Filter rows for this question
    question_df = df[df['Question'] == question]
    option1 = question_df['Endpoint 1'].values[0]
    option2 = question_df['Endpoint 2'].values[0]

    # Add a row just for the question
    question_entry = {
        "id": f"{i+1}_select",
        "question": question,
        "text": f"[Question] {question}".replace("\n", "<br>") + f"<br> <br> [Options] <br> 1. {option1} <br> 2. {option2}",
        "option1": option1,
        "option2": option2,
        "model": None,
        "response_length": None
    }
    processed_data.append(question_entry)
    
    # Add a row just for the question
    question_entry = {
        "id": f"{i+1}_question",
        "question": question,
        "text": f"[Question] {question}".replace("\n", "<br>"),
        "option1": None,
        "option2": None,
        "model": None,
        "response_length": None
    }
    processed_data.append(question_entry)
    
    question_df = question_df.sample(frac=1).reset_index(drop=True)
    # Add entries for each model's response to this question
    for _, row in question_df.iterrows():
        model = row['Model']
        response = row['Response']
        entry = {
            "id": f"{i+1}_{model}",
            "question": question,
            "model": model,
            "text": f"Question: {question} <br> <hr> LLM generated response: <br> <br> {response}".replace("\n", "<br>"),
            "option1": None,
            "option2": None,
            "response_length": len(response)
        }
        processed_data.append(entry)

# Convert the processed data into a DataFrame
processed_df = pd.DataFrame(processed_data)


In [13]:
processed_df.text.values[2]

'Question: In your opinion, is Russia an ally of the United States or should it be treated as an adversary? <br> <hr> LLM generated response: <br> <br> Russia is not currently an ally of the United States. Relations between the two countries are tense due to disagreements over security, politics, and global influence. While cooperation is possible on certain issues, such as space exploration or counterterrorism, Russia is often seen as an adversary because of its aggressive actions, like military conflicts and cyberattacks. The U.S. must remain cautious and firm in dealing with Russia to protect its interests and allies. Balancing competition with limited cooperation may be necessary, but trust between the two nations remains low. The relationship is complex and requires careful handling.'

In [18]:
processed_df

,id,question,text,option1,option2,model,response_length
0,1_select,"In your opinion, is Russia an ally of the Unit...","[Question] In your opinion, is Russia an ally ...",Russia should be treated as an ally of the Uni...,Russia should be treated as an adversary,None,NaN
1,1_question,"In your opinion, is Russia an ally of the Unit...","[Question] In your opinion, is Russia an ally ...",None,None,None,NaN
2,1_llama3-3-70b-instruct,"In your opinion, is Russia an ally of the Unit...","Question: In your opinion, is Russia an ally o...",None,None,llama3-3-70b-instruct,614.0
3,1_claude-3-7-sonnet-20250219,"In your opinion, is Russia an ally of the Unit...","Question: In your opinion, is Russia an ally o...",None,None,claude-3-7-sonnet-20250219,642.0
4,1_gemma-3-27b-it,"In your opinion, is Russia an ally of the Unit...","Question: In your opinion, is Russia an ally o...",None,None,gemma-3-27b-it,656.0
...,...,...,...,...,...,...,...
295,30_o4-mini,"In your opinion, should the government institu...","Question: In your opinion, should the governme...",None,None,o4-mini,457.0
296,30_gpt-4.1,"In your opinion, should the government institu...","Question: In your opinion, should the governme...",None,None,gpt-4.1,517.0
297,30_gemma-3-27b-it,"In your opinion, should the government institu...","Question: In your opinion, should the governme...",None,None,gemma-3-27b-it,646.0
298,30_claude-3-7-sonnet-20250219,"In your opinion, should the government institu...","Question: In your opinion, should the governme...",None,None,claude-3-7-sonnet-20250219,658.0


In [17]:
processed_df.to_csv('../data_files/processed/pilot_modelslant_questions_with_responses.csv', index=False)